In [1]:
class OneLine():
    def __init__(self,min_max_y) -> None:
        self.min_y = min_max_y[0]
        self.max_y = min_max_y[1]
        self.line = []
        self.beat = []
        self.sfn = []
        self.beam =[]
        self._line_len_ = len(self.line)
        self.clef_width = ()
        self.key_sfn = []
        self.accidental_sfn = []
    
    def decide_inside(self, symbol_or_note):
        if self.min_y<=symbol_or_note[2]<self.max_y: return True

    def oneline_append_symbol(self, symbol):
        if self.clef_width[0]<=symbol[1]<=self.clef_width[1]: return # 음자리표랑 겹치면 잘못 탐지한 것이므로 append 시키지 않음.

        if symbol[0] in set(range(8,19)):   ## 줄 맨 앞 박자
            self.beat.append(symbol)

        elif symbol[0] in set(range(20,28)): ## sfn
            self.sfn.append(symbol)
    
        else : self.line.append(symbol)


    def oneline_append_note(self,note):
        if self.clef_width[0]<=note[1]<=self.clef_width[1]: return # 음자리표랑 겹치면 잘못 탐지한 것이므로 append 시키지 않음.

        if note[0] in {15} :
            self.beam.append(note)
        else :
            self.line.append(note)

    def sort_line(self):
        new_line = []
        self.line = sorted(self.line, key = lambda x : x[1])
        for i in range(len(self.line)): #  merge
            if i == len(self.line)-1 :
                new_line.append(self.line[i])
                continue
            present = self.line[i]
            next = self.line[i+1]
            if abs(present[1]-next[1])<=3 and abs(present[2]-next[2])<=2:  # 라벨이 달라도 x,y좌표 차이가 다음것과 2이하이면 넘김
                continue
            new_line.append(self.line[i])
        self.line = new_line
    
    def decide_beat(self):
        if len(self.beat) != 2 :
            self.beat = None
        else:
            self.beat = sorted(self.beat, key=lambda x : x[2])
            upbeat = self.beat[0][0]-8
            downbeat = self.beat[1][0]-8
            self.beat = (upbeat,downbeat)

    def __len__(self):
        return self._line_len_
    

In [2]:
class Note():
    def __init__(self,note, pitch) -> None:
        self.label: int = note[0]
        self.bbox : list = note[1:]
        self.pitch: int = pitch
        self.sharp: bool = False
        self.flat: bool = False
        self.natural: bool = False
        self.duration: int = 16*(2**(self.label))
        self.measure: int = None

class Rest():
    def __init__(self,rest) -> None:
        self.label: int = rest[0]
        self.bbox : list = rest[1:]
        self.duration: int = 2**(35-self.label)
        self.measure: int = None

In [ ]:
import cv2
import numpy as np 

from conversion import conversion
from conversion_oneline import conversion_oneline
from note_class import info
from pitch_detection_only_G import pitch_detection_only_G 
from merge import merge_bbox
from SFN_detection import sfn_detection
from noise_removal import noise_removal
from beat_detection import beat_detection
from measure_calculation import measure_calculation


NAME = '0206_1'
S_NAME = 'school_bell_processed.txt'

note_label_file = f'/opt/ml/model/runs/notehead/{NAME}/labels/{S_NAME}'
symbol_label_file = f'/opt/ml/model/runs/symbols/{NAME}/labels/{S_NAME}'
img_path = '/opt/ml/model/source/school_bell.jpg'
staves = '/opt/ml/model/staves.txt'


## staff line 읽어오기 
original = cv2.imread(img_path) 
staves = open(staves, 'r')
staff_line = []
while True: 
    line = staves.readline() 
    if not line: 
        break
    staff_line.append(int(line.replace('\n', ''))) 
staves.close() 


## conversion 
clef_bbox, clef = conversion_oneline('clef', symbol_label_file, img_path, staff_line)
time_bbox, time = conversion_oneline('time', symbol_label_file, img_path, staff_line)
note_pos, staff_line = conversion('note', note_label_file, img_path, staff_line)
# note pos = [which, [label, x, y, w, h]]
merged_note_pos = merge_bbox(note_pos, 'merged_note.jpg', viz=True, img=original)
# merged_note pos = [which, [label, x, y, w, h]]

sharp, flat, natural = conversion_oneline('SFN', symbol_label_file, img_path, staff_line) 
# sharp = [note, [label, x, y, w, h]]

flag_dot = conversion_oneline('beat', note_label_file, img_path, staff_line) 
merged_flag_dot = merge_bbox(flag_dot, 'merged_beat.jpg', viz=True, img=original)

rest = conversion_oneline('rest', symbol_label_file, img_path, staff_line) 


## noise removal 
symbols = []
if len(sharp) > 0: 
    symbols.append(sharp) 
if len(flat) > 0: 
    symbols.append(flat) 
if len(natural) > 0: 
    symbols.append(natural)
# print('symbols', symbols)
noise_removed_note_pos = noise_removal(merged_note_pos, symbols, clef_bbox, original)
# noise_removed_note_pos = [which, [label, x, y, w, h]]


## beat detection 
beats = beat_detection(merged_flag_dot, rest, noise_removed_note_pos, original)
# beats = [beat]


## combine note heads & rest info 
combined = noise_removed_note_pos + rest
combined.sort(key=lambda x: (x[0], x[1][1]))

n = []
for c in combined: 
    m = info()
    m.staff = c[0]
    m.label = c[1][0]
    m.bbox = c[1][1:]

    n.append(m) 

for i in range(len(beats)): 
    n[i].duration = beats[i] 


## measure calculation  
measure = measure_calculation(beats, time, combined, original)
# measure = [measure]

for i in range(len(beats)): 
    n[i].measure = measure[i] 


## pitch detection 
pitches = pitch_detection_only_G(combined, staff_line, flat=False, original=original, viz=True) 
# pitches = [which, [label, x, y, w, h], pitch]


## sfn detection 
pitches_sfn = sfn_detection(sharp, flat, natural, pitches, original)
# pitches_sfn = [which, [label, x, y, w, h], pitch, sig여부]

for i, p in enumerate(pitches_sfn): 
    pos_pitch, key_sign = p 
    if pos_pitch[1] == 'quarter_rest' or pos_pitch[1] == '8th_rest': 
        continue 
    n[i].pitch = pos_pitch[1]
    if key_sign == 'sharp': 
        n[i].sharp = True 
    elif key_sign == 'flat': 
        n[i].flat = True 


In [4]:
def pitch(note):
  lst = ['E','D','C'] + ['B','A','G','F','E','D','C']*3
  return lst[int(note.pitch*2)]


def alter(note):                             #sfn 여부
  if note.sharp : return '1'
  elif note.flat: return '-1'
  else: return '0'


def note_type(note):
  if note.duration in {64,96}: return 'whole'
  elif note.duration in {32,48}: return 'half'
  elif note.duration in {16,24}: return 'quarter'
  elif note.duration in {8,12}: return 'eighth'
  elif note.duration in {4,6}: return '16th'
  elif note.duration in {2,3}: return '32nd'

def octave(note):
  if 0<=note.pitch<=1 : return '6'
  elif 1.5<=note.pitch<=4.5 : return '5'
  elif 5<=note.pitch<=8: return '4'
  else : return '3'


def write_note(note):
  global XML
  global TAB

    
  XML += TAB*'  ' + '<' + 'note' + '>' + '\n'   #note open
  TAB += 1
  XML += TAB*'  '+'<pitch>'+'\n'                #pitch open
  TAB += 1
  XML += TAB*'  '+'<step>'+pitch(note)+'</step>'+'\n' #step
  XML += TAB*'  ' + '<alter>'+alter(note)+'</alter>'+'\n'             #alter
  XML += TAB*'  ' + '<octave>'+octave(note)+'</octave>\n'           #octave
  TAB -= 1
  XML += TAB*'  '+'</pitch>'+'\n'                    #pitch close
  XML += TAB*'  '+'<duration>' + str(note.duration) + '</duration>' + '\n' #duration
  XML += TAB*'  '+'<type>' + note_type(note) + '</type>' + '\n' #type
  if note.duration%3 == 0: XML += TAB*'  ' + '<dot/>'   #dot
  XML += TAB*'  '+'<staff>1</staff>\n'
  XML += TAB*'  '+'<voice>1</voice>\n'
  TAB -= 1
  XML += TAB*'  ' + '<' + '/note' + '>' + '\n'

def write_rest(rest):
  global XML
  global TAB
  XML += TAB*'  ' + '<' + 'note' + '>' + '\n'   #note open
  TAB += 1
  XML += TAB*'  ' + '<rest measure="yes"/>'+ '\n'
  XML += TAB*'  ' + '<duration>' + str(rest.duration) + '</duration>' + '\n'
  TAB -= 1
  XML += TAB*'  ' + '<' + '/note' + '>' + '\n'


XML = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE score-partwise PUBLIC "-//Recordare//DTD MusicXML 4.0 Partwise//EN" "http://www.musicxml.org/dtds/partwise.dtd">
<score-partwise version="4.0">
  <work>
    <work-title>0_ori_ring</work-title>
  </work>
  <identification>
    <creator type="composer">Transcribed by Oemer</creator>
  </identification>
  <part-list>
    <score-part id="P1">
      <part-name>Piano</part-name>
      <score-instrument id="P1-I1">
        <instrument-name>Piano</instrument-name>
        <instrument-sound>keyboard.piano</instrument-sound>
      </score-instrument>
      <midi-instrument id="P1-I1">
        <midi-channel>1</midi-channel>
        <midi-program>1</midi-program>
        <volume>80</volume>
        <pan>0</pan>
      </midi-instrument>
    </score-part>
  </part-list>
  <part id="P1">
    <sound tempo="90"/>
    <measure number="1">
      <attributes>
        <divisions>16</divisions>
        <key>
          <fifths>0</fifths>
        </key>
        <staves>1</staves>
        <clef number="1">
          <sign>G</sign>
          <line>2</line>
        </clef>
      </attributes>
'''
TAB = 3

for m in n: 
  if m.label == 'rest': 
    write_rest(m)
  else: 
    write_note(m) 

XML+='''    </measure>
  </part>
</score-partwise>'''


print(XML)

<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE score-partwise PUBLIC "-//Recordare//DTD MusicXML 4.0 Partwise//EN" "http://www.musicxml.org/dtds/partwise.dtd">
<score-partwise version="4.0">
  <work>
    <work-title>0_ori_ring</work-title>
  </work>
  <identification>
    <creator type="composer">Transcribed by Oemer</creator>
  </identification>
  <part-list>
    <score-part id="P1">
      <part-name>Piano</part-name>
      <score-instrument id="P1-I1">
        <instrument-name>Piano</instrument-name>
        <instrument-sound>keyboard.piano</instrument-sound>
      </score-instrument>
      <midi-instrument id="P1-I1">
        <midi-channel>1</midi-channel>
        <midi-program>1</midi-program>
        <volume>80</volume>
        <pan>0</pan>
      </midi-instrument>
    </score-part>
  </part-list>
  <part id="P1">
    <sound tempo="90"/>
    <measure number="1">
      <attributes>
        <divisions>16</divisions>
        <key>
          <fifths>0</fifths>
        </key>
       

In [5]:
def pitch(note):
  lst = ['E','D','C'] + ['B','A','G','F','E','D','C']*3
  return lst[int(note.pitch*2)]


def alter(note):                             #sfn 여부
  if note.sharp : return '1'
  elif note.flat: return '-1'
  else: return '0'


def note_type(note):
  if note.duration in {64,96}: return 'whole'
  elif note.duration in {32,48}: return 'half'
  elif note.duration in {16,24}: return 'quarter'
  elif note.duration in {8,12}: return 'eighth'
  elif note.duration in {4,6}: return '16th'
  elif note.duration in {2,3}: return '32nd'

def octave(note):
  if 0<=note.pitch<=1 : return '6'
  elif 1.5<=note.pitch<=4.5 : return '5'
  elif 5<=note.pitch<=8: return '4'
  else : return '3'


def write_note(note):
  global XML
  global TAB

    
  XML += TAB*'  ' + '<' + 'note' + '>' + '\n'   #note open
  TAB += 1
  XML += TAB*'  '+'<pitch>'+'\n'                #pitch open
  TAB += 1
  XML += TAB*'  '+'<step>'+pitch(note)+'</step>'+'\n' #step
  XML += TAB*'  ' + '<alter>'+alter(note)+'</alter>'+'\n'             #alter
  XML += TAB*'  ' + '<octave>'+octave(note)+'</octave>\n'           #octave
  TAB -= 1
  XML += TAB*'  '+'</pitch>'+'\n'                    #pitch close
  XML += TAB*'  '+'<duration>' + str(note.duration) + '</duration>' + '\n' #duration
  XML += TAB*'  '+'<type>' + note_type(note) + '</type>' + '\n' #type
  if note.duration%3 == 0: XML += TAB*'  ' + '<dot/>'   #dot
  XML += TAB*'  '+'<staff>1</staff>\n'
  XML += TAB*'  '+'<voice>1</voice>\n'
  TAB -= 1
  XML += TAB*'  ' + '<' + '/note' + '>' + '\n'

def write_rest(rest):
  global XML
  global TAB
  XML += TAB*'  ' + '<' + 'note' + '>' + '\n'   #note open
  TAB += 1
  XML += TAB*'  ' + '<rest measure="yes"/>'+ '\n'
  XML += TAB*'  ' + '<duration>' + str(rest.duration) + '</duration>' + '\n'
  TAB -= 1
  XML += TAB*'  ' + '<' + '/note' + '>' + '\n'


XML = '''<?xml version="1.0" encoding="UTF-8"?>
<!DOCTYPE score-partwise PUBLIC "-//Recordare//DTD MusicXML 4.0 Partwise//EN" "http://www.musicxml.org/dtds/partwise.dtd">
<score-partwise version="4.0">
  <work>
    <work-title>0_ori_ring</work-title>
  </work>
  <identification>
    <creator type="composer">Transcribed by Oemer</creator>
  </identification>
  <part-list>
    <score-part id="P1">
      <part-name>Piano</part-name>
      <score-instrument id="P1-I1">
        <instrument-name>Piano</instrument-name>
        <instrument-sound>keyboard.piano</instrument-sound>
      </score-instrument>
      <midi-instrument id="P1-I1">
        <midi-channel>1</midi-channel>
        <midi-program>1</midi-program>
        <volume>80</volume>
        <pan>0</pan>
      </midi-instrument>
    </score-part>
  </part-list>
  <part id="P1">
    <sound tempo="90"/>
    <measure number="1">
      <attributes>
        <divisions>16</divisions>
        <key>
          <fifths>0</fifths>
        </key>
        <staves>1</staves>
        <clef number="1">
          <sign>G</sign>
          <line>2</line>
        </clef>
      </attributes>
'''
TAB = 3

for oneline in OneLineList:
    for note_or_rest in oneline.line:
        if type(note_or_rest) == Note : write_note(note_or_rest)
        elif type(note_or_rest) == Rest : write_rest(note_or_rest)

XML+='''    </measure>
  </part>
</score-partwise>'''


print(XML)

NameError: name 'OneLineList' is not defined

In [ ]:
a= 3
if a == 1 : print(1)
elif a == 2: print(3)